In [1]:
import csv
from collections import namedtuple
from collections import Counter
from collections import OrderedDict

In [2]:
open('employment.csv', 'r').readlines()[:5]

['employer,department,employee_id,ssn\n',
 'Stiedemann-Bailey,Research and Development,29-0890771,100-53-9824\n',
 'Nicolas and Sons,Sales,41-6841359,101-71-4702\n',
 'Connelly Group,Research and Development,98-7952860,101-84-0356\n',
 'Upton LLC,Marketing,56-9817552,104-22-0928\n']

In [3]:
open('personal_info.csv', 'r').readlines()[:5]

['ssn,first_name,last_name,gender,language\n',
 '100-53-9824,Sebastiano,Tester,Male,Icelandic\n',
 '101-71-4702,Cayla,MacDonagh,Female,Lao\n',
 '101-84-0356,Nomi,Lipprose,Female,Yiddish\n',
 '104-22-0928,Justinian,Kunzelmann,Male,Dhivehi\n']

In [4]:
open('update_status.csv', 'r').readlines()[:5]

['ssn,last_updated,created\n',
 '100-53-9824,2017-10-07T00:14:42Z,2016-01-24T21:19:30Z\n',
 '101-71-4702,2017-01-23T11:23:17Z,2016-01-27T04:32:57Z\n',
 '101-84-0356,2017-10-04T11:21:30Z,2016-09-21T23:04:07Z\n',
 '104-22-0928,2017-03-28T12:38:29Z,2016-04-15T11:37:17Z\n']

In [5]:
open('vehicles.csv', 'r').readlines()[:5]

['ssn,vehicle_make,vehicle_model,model_year\n',
 '100-53-9824,Oldsmobile,Bravada,1993\n',
 '101-71-4702,Ford,Mustang,1997\n',
 '101-84-0356,GMC,Yukon,2005\n',
 '104-22-0928,Oldsmobile,Intrigue,2000\n']

In [6]:
# Build namedtuples with fields corresponding to the data columns of files. Note 'ssn' is repetitive and is the common link among these separate individual files.

Employee = namedtuple('Employee', ('employer', 'department', 'employee_id', 'ssn'))
PersonalInfo = namedtuple('PersonalInfo', ('ssn', 'first_name', 'last_name', 'gender', 'language'))
UpdateStatus = namedtuple('UpdateStatus', ('ssn', 'last_updated', 'created'))
Vehicles = namedtuple('Vehicles', ('ssn', 'vehicle_make', 'vehicle_model', 'model_year'))

In [7]:
# This is to show how to build a generator to yield a namedtuple with data from the relevant file (here, 'employment.csv')

def get_employee():
    with open('employment.csv', 'r') as emp:
        emp.readline() # ignore the header
        reader = csv.reader(emp, delimiter=',')
        for item in reader:
            yield Employee(*item)

In [8]:
# define generator to get data from function via next() operator

employee_gen = get_employee()

In [9]:
next(employee_gen) # generator yielding next data

Employee(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824')

#### Goal 1: Build generators for each of the datasets (assuming data is aligned across all the 4 files with respect to the field variable 'ssn')

In [10]:
# We can build a common function to build generator yielding data from all files

# Arguments 'file' and 'tup_name' correspond to respective filename and the namedtuple (defined to store data from that file).
# This common file is named as 'employee_data' as it stores all employee related data

def employee_data(file, tup_name):
    with open(file, 'r') as f:
        f.readline() # ignore the header
        reader = csv.reader(f, delimiter=',')
        for item in reader:
            yield tup_name(*item)

In [11]:
# redo the employee_gen generated above (and check we got the same result)
employee_generator = employee_data('employment.csv', Employee)

In [12]:
next(employee_generator)

Employee(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824')

In [13]:
# Getting Personal Info of the employee

pin_gen = employee_data('personal_info.csv', PersonalInfo)

In [14]:
next(pin_gen)

PersonalInfo(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')

In [15]:
# Define UpdateStatus data generator

sts_gen = employee_data('update_status.csv', UpdateStatus)

In [16]:
next(sts_gen)

UpdateStatus(ssn='100-53-9824', last_updated='2017-10-07T00:14:42Z', created='2016-01-24T21:19:30Z')

In [17]:
# Define vehicle info of employees

vhs_gen = employee_data('vehicles.csv', Vehicles)

In [18]:
next(vhs_gen)

Vehicles(ssn='100-53-9824', vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year='1993')

In [19]:
# We can also get all the iterators in single yield from multiple generators at once, like this

def get_data():
    with open('employment.csv', 'r') as emp, open('personal_info.csv', 'r') as pin, open('update_status.csv', 'r') as sts, open('vehicles.csv', 'r') as vhs:
        bad_data = f'DataFeedError: Data not matching; need clean-up or sorting based on "ssn"'
        c_emp = emp.readline().strip().split(',')
        c_pin = pin.readline().strip().split(',')
        c_sts = sts.readline().strip().split(',')
        c_vhs = vhs.readline().strip().split(',')
        
        rdr_emp = csv.reader(emp, delimiter=',')
        rdr_pin = csv.reader(pin, delimiter=',')
        rdr_sts = csv.reader(sts, delimiter=',')
        rdr_vhs = csv.reader(vhs, delimiter=',')
        
        for i, j, k, l in zip(rdr_emp, rdr_pin, rdr_sts, rdr_vhs):
            if i[3] == j[0] == k[0] == l[0]:
                # yield i, j, k, l
                yield Employee(*i), PersonalInfo(*j), UpdateStatus(*k), Vehicles(*l)
                # yield from (Employee(*next(rdr_emp)), PersonalInfo(*next(rdr_pin)), UpdateStatus(*next(rdr_sts)), Vehicles(*next(rdr_vhs)))
            else:
                yield bad_data

In [20]:
data_gen = iter(get_data())

In [21]:
next(data_gen)

(Employee(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824'),
 PersonalInfo(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic'),
 UpdateStatus(ssn='100-53-9824', last_updated='2017-10-07T00:14:42Z', created='2016-01-24T21:19:30Z'),
 Vehicles(ssn='100-53-9824', vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year='1993'))

#### Goal 2: Create a single iterable that combines all the columns from all the iterators.

Member is the namedtuple that carries all the field variables present disjointly in individual dataset of 4 files supplied. The order of fields were slightly modified to represent the data in a better format i.e., member's first name and last name comes before employment details. The field 'ssn' is kept as a first field, just in case, if we were required to use a dictionary later, we can just use the first field (0), without having to count for index value. If the data coming from files are not synchronized with respect to the field 'ssn', then function yields feed data error.

In [22]:
Member = namedtuple('Member', ('ssn', 'first_name', 'last_name', 'gender', 'employer', 'department', 'employee_id', 'language', 'vehicle_make', 'vehicle_model', 'model_year', 'created', 'last_updated'))

def person_data():
    for item in get_data():
        bad_data = f'DataFeedError: Data not matching; need clean-up or sorting based on "ssn"'
        if isinstance(item, tuple):
            i, j, k, l = item
            if i.ssn == j.ssn == k.ssn == l.ssn:
                yield Member(i.ssn, j.first_name, j.last_name, j.gender, i.employer, i.department, i.employee_id, j.language, l.vehicle_make, l.vehicle_model, l.model_year, k.created, k.last_updated)
        else:
            yield bad_data

In [23]:
person = person_data()

In [24]:
next(person)

Member(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', language='Icelandic', vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year='1993', created='2016-01-24T21:19:30Z', last_updated='2017-10-07T00:14:42Z')

In [25]:
# This function outputs the next 5 persons from the person_data().

def get_next_5_persons(start=0, end=5):
    n = 0
    person = person_data()
    for item in person:
        n += 1
        # print(f'n = {n}')
        if n > start:
            print(item,'\n')
        if n >= end:
            break       

In [26]:
get_next_5_persons(start=0, end=5)

Member(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', language='Icelandic', vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year='1993', created='2016-01-24T21:19:30Z', last_updated='2017-10-07T00:14:42Z') 

Member(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', employer='Nicolas and Sons', department='Sales', employee_id='41-6841359', language='Lao', vehicle_make='Ford', vehicle_model='Mustang', model_year='1997', created='2016-01-27T04:32:57Z', last_updated='2017-01-23T11:23:17Z') 

Member(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', employer='Connelly Group', department='Research and Development', employee_id='98-7952860', language='Yiddish', vehicle_make='GMC', vehicle_model='Yukon', model_year='2005', created='2016-09-21T23:04:07Z', last_updated='2017-10-04T11:21:30Z') 

Member(ssn='1

#### Goal 3: Identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the last_updated field from the status_update file.

In [27]:
def current_members():
    from datetime import datetime
    for item in person_data():
        update_date = item.last_updated
        create_date = item.created
        item = item._replace(
            last_updated = datetime.strptime(update_date, '%Y-%m-%dT%H:%M:%SZ'),
            created = datetime.strptime(create_date, '%Y-%m-%dT%H:%M:%SZ'))
        if item.last_updated > datetime(2017, 1, 3):
            yield item 

In [28]:
cur_members = current_members()

In [29]:
next(cur_members)

Member(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', language='Icelandic', vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year='1993', created=datetime.datetime(2016, 1, 24, 21, 19, 30), last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42))

#### Goal 4: Find the largest group of car makes for each gender.

Possibly more than one such group per gender exists (equal sizes).

In [30]:
def carmake_per_gender(func, gender):
    data = [Member._make(member).vehicle_make for member in func() if member.gender==gender]
    # print(len(data))
    return Counter(data)

In [31]:
cars_by_female = carmake_per_gender(current_members, 'Female')

In [32]:
cars_by_male = carmake_per_gender(current_members, 'Male')

In [33]:
# print 3 most common carmakes owned by female members

cars_by_female.most_common(3)

[('Ford', 48), ('Chevrolet', 47), ('Mitsubishi', 25)]

In [34]:
# print 3 most common carmakes owned by male members

cars_by_male.most_common(3)

[('Ford', 44), ('Chevrolet', 38), ('GMC', 31)]

Ford is the largest carmake of the cars owned by male and female members.

In [35]:
male_cars = set(list(cars_by_male.keys()))
female_cars = set(list(cars_by_female.keys()))

In [36]:
male_cars - female_cars # carmakes owned by male but not by female

{'Aptera', 'Corbin', 'Daewoo', 'Hummer', 'Jensen', 'Maserati', 'Maybach'}

In [37]:
female_cars - male_cars # carmakes owned by female but not by male

{'Bugatti', 'Morgan'}